<a href="https://colab.research.google.com/github/farenhai/github-demo-cong-necro/blob/main/Slide_ISA_QuerySpeedTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faker pandas
!pip install psutil

In [4]:
import sqlite3
import time
import random
from faker import Faker
import psutil

# Initialize Faker
fake = Faker()

# Record size per child type
records_per_child = 100000

# Create a new in-memory SQLite database connection
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Base Table
cursor.execute('''
CREATE TABLE Employee (
    EmployeeID INT PRIMARY KEY,
    FullName TEXT NOT NULL,
    Gender TEXT,
    PhoneNumber TEXT,
    Address TEXT,
    EmployeeType TEXT NOT NULL
)
''')
# Add index on EmployeeID for Employee table
cursor.execute('CREATE INDEX idx_employee_id ON Employee (EmployeeID)')

# Part-Time
cursor.execute('''
CREATE TABLE PartTime (
    EmployeeID INT PRIMARY KEY REFERENCES Employee(EmployeeID),
    HoursWorked INT,
    HourlyRate REAL
)
''')
# Add index on EmployeeID for PartTime table
cursor.execute('CREATE INDEX idx_parttime_id ON PartTime (EmployeeID)')

# Full-Time
cursor.execute('''
CREATE TABLE FullTime (
    EmployeeID INT PRIMARY KEY REFERENCES Employee(EmployeeID),
    SeniorityBonus REAL,
    DaysWorked INT
)
''')
# Add index on EmployeeID for FullTime table (if needed for other queries)

# Internship
cursor.execute('''
CREATE TABLE Internship (
    EmployeeID INT PRIMARY KEY REFERENCES Employee(EmployeeID),
    InternshipAllowance REAL,
    InternshipDuration TEXT
)
''')
# Add index on EmployeeID for Internship table (if needed for other queries)

# Freelancer
cursor.execute('''
CREATE TABLE Freelancer (
    EmployeeID INT PRIMARY KEY REFERENCES Employee(EmployeeID),
    FreelanceFee REAL,
    Expertise TEXT
)
''')
# Add index on EmployeeID for Freelancer table
cursor.execute('CREATE INDEX idx_freelancer_id ON Freelancer (EmployeeID)')


# Insert Data into Separate Tables
print("Inserting data into Separate Tables...")
start_time = time.time()

for i in range(records_per_child):
    # Insert into Employee
    cursor.execute('''
        INSERT INTO Employee VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        i, fake.name(), random.choice(['M', 'F']), fake.phone_number(), fake.address(),
        'PartTime'  # Example: Start with PartTime type
    ))
    cursor.execute('''
        INSERT INTO PartTime VALUES (?, ?, ?)
    ''', (
        i, random.randint(10, 40), round(random.uniform(10, 50), 2)
    ))

for i in range(records_per_child, records_per_child * 2):
    cursor.execute('''
        INSERT INTO Employee VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        i, fake.name(), random.choice(['M', 'F']), fake.phone_number(), fake.address(),
        'FullTime'
    ))
    cursor.execute('''
        INSERT INTO FullTime VALUES (?, ?, ?)
    ''', (
        i, round(random.uniform(500, 2000), 2), random.randint(15, 30)
    ))

for i in range(records_per_child * 2, records_per_child * 3):
    cursor.execute('''
        INSERT INTO Employee VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        i, fake.name(), random.choice(['M', 'F']), fake.phone_number(), fake.address(),
        'Internship'
    ))
    cursor.execute('''
        INSERT INTO Internship VALUES (?, ?, ?)
    ''', (
        i, round(random.uniform(300, 500), 2), fake.date_this_decade()
    ))

for i in range(records_per_child * 3, records_per_child * 4):
    cursor.execute('''
        INSERT INTO Employee VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        i, fake.name(), random.choice(['M', 'F']), fake.phone_number(), fake.address(),
        'Freelancer'
    ))
    cursor.execute('''
        INSERT INTO Freelancer VALUES (?, ?, ?)
    ''', (
        i, round(random.uniform(500, 5000), 2), fake.bs()
    ))

conn.commit()
print(f"Separate Tables: {records_per_child * 4} records inserted in {time.time() - start_time:.2f} seconds.")

# --- SingleTable Schema ---
cursor.execute('''
CREATE TABLE SingleTable (
    EmployeeID INT PRIMARY KEY,
    FullName TEXT NOT NULL,
    Gender TEXT,
    PhoneNumber TEXT,
    Address TEXT,
    EmployeeType TEXT,
    HoursWorked INT NULL,
    HourlyRate REAL NULL,
    SeniorityBonus REAL NULL,
    DaysWorked INT NULL,
    InternshipAllowance REAL NULL,
    InternshipDuration TEXT NULL,
    FreelanceFee REAL NULL,
    Expertise TEXT NULL
)
''')

# Import Data from Separate Tables into SingleTable
print("Importing data into SingleTable...")
start_time = time.time()

# PartTime
cursor.execute('''
INSERT INTO SingleTable (EmployeeID, FullName, Gender, PhoneNumber, Address, EmployeeType, HoursWorked, HourlyRate)
SELECT e.EmployeeID, e.FullName, e.Gender, e.PhoneNumber, e.Address, e.EmployeeType, p.HoursWorked, p.HourlyRate
FROM Employee e
JOIN PartTime p ON e.EmployeeID = p.EmployeeID
''')

# FullTime
cursor.execute('''
INSERT INTO SingleTable (EmployeeID, FullName, Gender, PhoneNumber, Address, EmployeeType, SeniorityBonus, DaysWorked)
SELECT e.EmployeeID, e.FullName, e.Gender, e.PhoneNumber, e.Address, e.EmployeeType, f.SeniorityBonus, f.DaysWorked
FROM Employee e
JOIN FullTime f ON e.EmployeeID = f.EmployeeID
''')

# Internship
cursor.execute('''
INSERT INTO SingleTable (EmployeeID, FullName, Gender, PhoneNumber, Address, EmployeeType, InternshipAllowance, InternshipDuration)
SELECT e.EmployeeID, e.FullName, e.Gender, e.PhoneNumber, e.Address, e.EmployeeType, i.InternshipAllowance, i.InternshipDuration
FROM Employee e
JOIN Internship i ON e.EmployeeID = i.EmployeeID
''')

# Freelancer
cursor.execute('''
INSERT INTO SingleTable (EmployeeID, FullName, Gender, PhoneNumber, Address, EmployeeType, FreelanceFee, Expertise)
SELECT e.EmployeeID, e.FullName, e.Gender, e.PhoneNumber, e.Address, e.EmployeeType, f.FreelanceFee, f.Expertise
FROM Employee e
JOIN Freelancer f ON e.EmployeeID = f.EmployeeID
''')

conn.commit()
print(f"SingleTable: {records_per_child * 4} records imported in {time.time() - start_time:.2f} seconds.")


# Complex Single Table Query
single_table_query = '''
SELECT
    EmployeeID,
    FullName,
    Gender,
    PhoneNumber,
    Address,
    HoursWorked,
    HourlyRate
FROM SingleTable
WHERE EmployeeType = 'PartTime'
  AND HoursWorked > 30
  AND HourlyRate > 25
ORDER BY HoursWorked DESC
'''

# Complex Separate Tables Query
separate_table_query = '''
SELECT
    e.EmployeeID,
    e.FullName,
    e.Gender,
    e.PhoneNumber,
    e.Address,
    pt.HoursWorked,
    pt.HourlyRate
FROM Employee e
JOIN PartTime pt ON e.EmployeeID = pt.EmployeeID
WHERE pt.HoursWorked > 30
  AND pt.HourlyRate > 25
ORDER BY pt.HoursWorked DESC
'''

# Test Single Table Query
print("Testing Complex Single Table Query...")
cpu_usage_before = psutil.cpu_percent(interval=None)
start_time = time.time()
cursor.execute(single_table_query)
single_table_result = cursor.fetchall()
elapsed_time_single = time.time() - start_time
cpu_usage_after = psutil.cpu_percent(interval=None)
cpu_usage_single = cpu_usage_after - cpu_usage_before
print(f"Single Table Query: {len(single_table_result)} results, Elapsed Time = {elapsed_time_single:.2f} seconds")

# Test Separate Tables Query
print("Testing Complex Separate Tables Query...")
cpu_usage_before = psutil.cpu_percent(interval=None)
start_time = time.time()
cursor.execute(separate_table_query)
separate_table_result = cursor.fetchall()
elapsed_time_separate = time.time() - start_time
cpu_usage_after = psutil.cpu_percent(interval=None)
cpu_usage_separate = cpu_usage_after - cpu_usage_before
print(f"Separate Tables Query: {len(separate_table_result)} results, Elapsed Time = {elapsed_time_separate:.2f} seconds")



Inserting data into Separate Tables...
Separate Tables: 400000 records inserted in 187.77 seconds.
Importing data into SingleTable...
SingleTable: 400000 records imported in 0.79 seconds.
Testing Complex Single Table Query...
Single Table Query: 20113 results, Elapsed Time = 0.19 seconds
Testing Complex Separate Tables Query...
Separate Tables Query: 20113 results, Elapsed Time = 0.08 seconds
